In [2]:
from langchain_community.llms import Ollama

llm = Ollama(model="llama3")

In [4]:
# load the docs
from langchain_community.document_loaders import TextLoader

loader = TextLoader("./files/about_nutrition.txt")
documents = loader.load()

In [7]:
# use chromadb for embedding 
from langchain_chroma import Chroma
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_text_splitters import CharacterTextSplitter

# split the document into chunks
text_splitter = CharacterTextSplitter(chunk_size = 1000, chunk_overlap = 0)
docs = text_splitter.split_documents(documents)

# define the embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# store chunks as embedding to vectorstore ie chromadb
db = Chroma.from_documents(docs, embedding_function)

/Users/amishchopra/.local/share/virtualenvs/trying_llms-tq-8R2qj/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/amishchopra/.local/share/virtualenvs/trying_llms-tq-8R2qj/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/Users/amishchopra/.local/share/virtualenvs/trying_llms-tq-8R2qj/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [8]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# take query from user and do a similarity seach in chromadb which will be our context
query = "Why is hydration important?"
context_from_docs = db.similarity_search(query)

# create context query prompt
prompt_template = PromptTemplate.from_template(
    """
        Context: {context}

        Question: {user_question}

        Instructions:

        Answer the user's question "{user_question}" using the information provided in the context "{context}".
        Ensure your answer is factually grounded in the context.
        If the context doesn't have enough information to answer the question, return "I don't have that knowledge".
        Optional:

        You can add additional information to the Instructions section depending on your specific needs.
        For example, you might specify the desired answer length or tone.
        Consider including a repetition of the instructions at the end for better model adherence.
        Example:

        Context:  The capital of France is Paris. Paris is a beautiful city with a rich history. It is famous for its landmarks like the Eiffel Tower and the Louvre Museum.

        Question: What is the capital of France?

        Instructions: Answer the user's question "What is the capital of France?" using the information provided in the context "The capital of France is Paris...". Ensure your answer is factually grounded in the context. If the context doesn't have enough information to answer the question, return "I don't have that knowledge".

        Answer: Paris

        """
)

parser = StrOutputParser()
chain = prompt_template | llm | parser

# chain.invoke({"context":context_from_docs, "user_question" : query})

# steaming the output 
async for chunk in chain.astream({"context":context_from_docs, "user_question" : query}):
    print(chunk, end='', flush=True)


Why is hydration important?

According to the provided context, water plays a vital role in digestion, nutrient absorption, waste removal, temperature regulation, and lubrication of joints. Dehydration, even mild, can lead to fatigue, headaches, and decreased cognitive function. Therefore, it is essential for maintaining a well-hydrated and healthy body, with a recommended daily goal of eight glasses of water.